# Domain: Transportation

## Topic: Autonomous vehicles

### Team: Optimax

We are testing algorithms for Autonomous vehicles. <br>
Using carla simulator to testing in virtural enviroment.<br>
The CARLA simulator consists of a scalable client-server architecture.

In [1]:
print("Hello world")

Hello world


First need to install carla 

In [ ]:
# !pip install carla

In [2]:
import tensorflow as tf
print(tf.__version__)

2.9.1


In [11]:
import carla
import math
import random
import time
import numpy as np
# import tensorflow as tf
import cv2
from collections import deque
from keras.applications.xception import Xception
from keras.layers import Dense, GlobalAveragePooling2D
from keras.optimizers import Adam
from keras.models import Model

Lets create a client and connect to carla local server

In [8]:
SHOW_PREVIEW = False
IMG_WIDTH = 640
IMG_HEIGHT = 480
SECONDS_PER_EPISODE = 10
REPLAY_MEMORY_SIZE = 5_000   #5,000
MIN_REPLAY_MEMORY_SIZE = 1_000
MINIBATCH_SIZE = 16
PREDICTION_BATCH_SIZE = 1
TRAINING_BATCH_SIZE = MINIBATCH_SIZE // 4
UPDATE_TARGET_EVERY = 5
MODEL_NAME = "Xception"

MEMORY_FRACTION = 0.8
MIN_REWARD = -200

EPISODES = 100

DISCOUNT = 0.99
epsilon = 1
EPSILON_DECAY = 0.95 #0.9975 99975
MIN_EPSILON = 0.001

AGGREGATE_STATS_EVERY = 10

In [4]:
class CarEnv:
    SHOW_CAM = SHOW_PREVIEW
    STEER_AMT = 1.0 
    img_width = IMG_WIDTH
    img_height = IMG_HEIGHT
    front_camera = None
    
    def __int__(self):
        try:
            self.client = carla.Client("localhose", 2000)
            print("connect with server")
        except Exception as e:
            print(e)
        self.client.set_timeout(3.0)
        self.world = self.client.get_world()
        self.blueprint_library = self.world.get_blueprint_library()
        self.model_3 = blueprint_library.filter("model3")[0]
        
    def reset(self):
        self.collision_hist = []
        self.actor_list = []
        
        self.transform = random.choice(self.world.get_map().get_spawn_points())
        self.vehicle = self.world.spawn_actor(self.model_3, self.transform)
        self.actor_list.append(self.vehicle)
        
        self.rgb_cam = self.blueprint_library.find('sensor.camera.rgb')
        self.rgb.set_attribute("image_size_x", f"{self.img_width}")
        self.rgb.set_attribute("image_size_y", f"{self.img_height}")
        self.rgb.set_attribute("fov", f"{110}")
        
        transform = carla.Transform(carla.Location(x=2.5, z=0.7))
        self.sensor = self.world.spawn_actor(self.rgb_cam, transform, attach_to = self.vehicle)
        self.actor_list.append(self.sensor)
        self.sensor.lissten(lambda data: self.process_img(data))
        
        self.vehicle.apply_control(carls.VehicleControl(throttle=0.0, brake=0.0))
        time.sleep(4)
        
        colsensor = self.blueprint_library.find("sensor.other.collision")
        self.colsensor = self.world.spawn_actor(colsensor, transform, attach_to=self.vehicle)
        self.actor_list.append(self.colcensor)
        self.colcensor.listen(lambda event: self.collision_data(event))
        
        while self.front_camera is None:
            time.sleep(0.01)
        
        self.episode_start = time.time()
        
        self.vehicle.apply_control(carla.VehicleControl(throttle=0.0, brake=0.0))
        
        return self.front_camera
    
    def collision_data(self, event):
        self.collision_hist.append(event)
        
    def process_img(self, image):
        i = np.array(image.raw_data)
        i2 = i.reshape((self.img_height, self.img_width, 4))
        i3 = i2[:, :, :3]
        if self.SHOW_CAM:
            cv2.imshow("", i3)
            cv2.waitKey(1)
#         return i3/255.0 
        self.front_camera = i3
    
    def step(self, action):
        if action == 0:
            self.vehicle.apple_control(carla.VehicleControl(throttle=1.0, steer = -1*self.STEER_AMT))
        elif action == 1:
            self.vehicle.apple_control(carla.VehicleControl(throttle=1.0, steer = 0))
        elif action == 2:
            self.vehicle.apple_control(carla.VehicleControl(throttle=1.0, steer = 1*self.STEER_AMT))
        
        v = self.vehicle.get_velocity()
        kmh = int(3.6 * math.sqrt(v.x**2 + v.y**2 + v.z**2))
        
        if len(self.collision_hist) != 0:
            done = True
            reward = -200
        elif kmh < 50:
            done = False
            reward = -1
        else:
            done = False
            reward = 1
        
        if self.episode_start + SECONDS_PER_EPISODE < time.time():
            done = True
        
        return self.front_camera, reward, done, None

In [22]:
class DQNAgent:
    def __init__(self):
        self.model = self.create_model()
        self.target_model = self.create_model()
        self.traget_model.set_weights(self.model.get_weights())
        
        self.replay_memory = deque(maxlen = REPLAY_MEMORY_SIZE)
        
        self.tensorboard = ModifiedTensorBoard(log_dir = f"logs/{MODEL_NAME}-{int(time.time())}")
        self.target_update_counter = 0
        self.graph = tf.get_default_graph()
        
        self.terminate = False
        self.last_logged_episode = 0
        self.training_initialized = False
    
    def create_model(self):
        base_model = Xception(weights=None, include_top=False, input_shape(IMG_HEIGHT, IMG_WIDTH, 3))
        
        x = base_model.output
        x = GlobalAveragePooling2D()(x)
        
        predictions = Dense(3, activation = "linear")(x)
        model = Model(inputs = base_model.input, outputs = predictions)
        model.compile(loss="mse", optimizer=Adam(lr=0.001),metrics=["accuracy"])
        return model
    
    def update_replay_memory(self, transition):
        self.replay_momory.append(transition)
    
    def train(self):
        if len(self.replay_memory) < MIN_REPLAY_MEMORY_SIZE:
            return
        minbatch = random.sample(self.replay_memory, MINIBATCH_SIZE)
        
        current_states = np.array([transition[0] for transition in minibatch])/255
        with self.graph.as_default():
            current_qs_list = self.model.predict(current_states, PREDICTION_BATCH_SIZE)
            
        new_current_states = np.array([transition[3] for transition in minibatch])/255
        with self.graph.as_default():
            future_qs_list = self.target_model.predict(new_current_states, PREDICTION_BATCH_SIZE)
        
        X = []
        y = []
        
        for index, (current_state, action, reward, new_state, done) in enumerate(minibatch):
            if not done:
                max_future_q = np.max(future_qs_list[index])
                new_q = reward + DISCOUNT * max_future_q
            else:
                new_q = reward
            
        current_qs = current_qs_list[index]
        current_qs[action] = new_q
        
        X.append(current_state)
        y.append(current_qs)
        
    log_this_step = False
    if self.tensorboard.step > self.last_logged_episode:
        log_this_step = True
        self.last_log_episode = self.tensorboard.step
    
    with self.graph.as_default():
        self.model.fit(np,array(X)/255, np.array(y), batch_size= TRAINING_BATCH_SIZE, werbose=0, shuffle=False, callbacks=[self.tensorboard] if log_this_step else None)
    
    if log_this_step:
        self.target_updat_counter += 1
        
    if self.target_update_counter > UPDATE_TARGET_EVERY:
        self.target_model.set_weights(self.model.get_weights())
        self.target_update_counter = 0
        
    def get_qs(self, state):
        return shelf.model.predict(np.array(state).reshape(-1 * state.shape)/255)[0]
    
    def train_in_loop(self):
        X = np.random.uniform(size=(1, IMG_HEIGHT, IMG_WIDTH, 3)).astype(np.float32)
        y = np.random.uniform(size=(1, 3)).astype(np.float32)
        with self.graph.as_default():
            self.model.fit(X,y, verbose= False, batch_size=1)
        
        self.training_initialized = True
        
        while Ture:
            if self.terminate:
                return
            self.train()
            time.sleep(0.01)
            

SyntaxError: positional argument follows keyword argument (2637358257.py, line 18)

In [ ]:
# try:
#     client = carla.Client('localhost', 2000)
#     client.set_timeout(5.0)
#     world = client.get_world()
#     print("connected with Server")
# except Exception as e:
#     print("An exception occurred:")
#     print(e)

In [ ]:
# blueprint_library = world.get_blueprint_library()
# spawn_points = world.get_map().get_spawn_points()

Lets create some vehicles for the city and make it as someone is driving it 

# NPC for City

In [ ]:
# npc_list = []
# actor_list = [] 

In [ ]:
# for i in range(10):
#     vehicle_bp = random.choice(blueprint_library.filter('vehicle'))
#     try:
#         npc = world.spawn_actor(vehicle_bp, random.choice(spawn_points))
#         npc_list.append(npc)
#     except:
#         print("one vehicle not spawn!")

In [ ]:
# for v in world.get_actors().filter('*vehicle*'):
#     v.set_autopilot(True)

# Lets start our model

Create a vehicle and locate its location

In [ ]:
# for a in range(50):
#     try:
#         vehicle = world.spawn_actor(
#             blueprint_library.find(
#                 random.choice([
#                     'vehicle.dogde.charger_2020',
#                     'vehicle.ford.mustang',
#                     'vehicle.mercedes.coupe_2020',
#                     'vehicle.tesla.model3'
#                 ])
#             ),
#             random.choice(spawn_points)
#         )
#         actor_list.append(vehicle)
#         print("selected")
#         break
#     except:
#         print("Next vehicle")

In [ ]:
# spectator = world.get_spectator()
# transform = carla.Transform(vehicle.get_transform().transform(carla.Location(x=-4, z=2.5)), vehicle.get_transform().rotation)
# spectator.set_transform(transform)

In [ ]:
# for i in actor_list:
#     print(i)

In [ ]:
# Run above

In [ ]:
# vehicle.set_autopilot(True)

## Control of sensors 

In [ ]:
# IMG_WIDTH = 640
# IMG_HEIGHT = 480

In [ ]:
# camera_bp = blueprint_library.find('sensor.camera.rgb')
# camera_bp.set_attribute('image_size_x', str(IMG_WIDTH))
# camera_bp.set_attribute('image_size_y', str(IMG_HEIGHT))
# camera_bp.set_attribute("fov", "110")

# camera_init_trans = carla.Transform(carla.Location(x = 2.5, z=2.0))
# camera = world.spawn_actor(camera_bp, camera_init_trans, attach_to = vehicle)
# actor_list.append(camera)

In [ ]:
# def process_img(image):
#     i = np.array(image.raw_data)
#     i2 = i.reshape((IMG_HEIGHT, IMG_WIDTH, 4))
#     i3 = i2[:, :, :3]
#     cv2.imshow("", i3)
#     cv2.waitKey(1)
#     return i3/255.0


# try:
#     camera.listen(lambda data: process_img(data))
#     time.sleep(10)
#     camera.destroy()
# except Exception as e:
#     print(e)

In [ ]:
# try:
#     for actor in actor_list:
#         actor.destroy()
#     print("All actors cleaned")
# except Exception as e:
#     print(e)

In [ ]:
# try:
#     for npc in npc_list:
#         npc.destroy()
#     print("All NPC cleaned")
# except Exception as e: 
#     print(e)